In [1]:
!nvidia-smi

import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

!pip install -U transformers datasets evaluate accelerate sacrebleu rouge_score py7zr sentencepiece



Wed Jan  7 00:12:17 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install "pydantic<2.12" --upgrade
!pip install "gradio<6" --upgrade
!pip install "SQLAlchemy>=1.4,<3" --upgrade
!pip uninstall -y xmanager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 12.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.7 MB/s eta 0:00:0000:01
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.41.5
    Uninstalling pydantic_core-2.41.5:
      Successfully uninstalled pydantic_core-2.41.5
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.12.5
    Uninstalling pydantic-2.12.5:
      Successfully uninstalled pydantic-2.12.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires SQLAlchemy<3,>=1.4, but you have sqlalchemy 1.2.19 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 MB 30.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [4]:
import torch, numpy as np, nltk, evaluate
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq, TrainingArguments, Trainer
)

nltk.download("punkt")
device = "cuda" if torch.cuda.is_available() else "cpu"
device


E0000 00:00:1767744810.397451      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767744810.461562      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767744810.996713      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767744810.996748      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767744810.996751      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767744810.996754      55 computation_placer.cc:177] computation placer already registered. Please check linka

'cuda'

In [5]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [6]:
dataset = load_dataset("knkarthick/samsum")


README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14731 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

In [7]:
def preprocess(batch):
    inputs = tokenizer(batch["dialogue"], max_length=1024, truncation=True)
    outputs = tokenizer(batch["summary"], max_length=128, truncation=True)

    batch["input_ids"] = inputs["input_ids"]
    batch["attention_mask"] = inputs["attention_mask"]
    batch["labels"] = outputs["input_ids"]
    return batch

dataset = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)



Map:   0%|          | 0/14731 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [8]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [9]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    preds = tokenizer.batch_decode(preds, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    scores = rouge.compute(predictions=preds, references=labels)

    return {
        "rouge1": scores["rouge1"],
        "rouge2": scores["rouge2"],
        "rougeL": scores["rougeL"]
    }


In [10]:
training_args = TrainingArguments(
    output_dir="pegasus-samsum",

    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,

    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,

    fp16=True,
    warmup_ratio=0.06,
    lr_scheduler_type="linear",

    eval_strategy="steps",
    eval_steps=2000,                # ⬅ less frequent eval
    logging_steps=200,
    save_strategy="steps",
    save_steps=2000,
    save_total_limit=1,

    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,

    prediction_loss_only=False,
    eval_accumulation_steps=1,      # ⬅ THIS is the OOM fix

    report_to="none"
)


In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/tmp/ipykernel_55/3923227464.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=1383, training_loss=1.6812124300589186, metrics={'train_runtime': 13625.4592, 'train_samples_per_second': 3.243, 'train_steps_per_second': 0.102, 'total_flos': 2.3063041844699136e+16, 'train_loss': 1.6812124300589186, 'epoch': 3.0})

In [13]:
model.save_pretrained("pegasus-samsum-model")
tokenizer.save_pretrained("pegasus-samsum-tokenizer")


('pegasus-samsum-tokenizer/tokenizer_config.json',
 'pegasus-samsum-tokenizer/special_tokens_map.json',
 'pegasus-samsum-tokenizer/spiece.model',
 'pegasus-samsum-tokenizer/added_tokens.json',
 'pegasus-samsum-tokenizer/tokenizer.json')

In [14]:
from transformers import pipeline
import torch

pipe = pipeline(
    "summarization",
    model="./pegasus-samsum-model",
    tokenizer="./pegasus-samsum-tokenizer",
    device=0 if torch.cuda.is_available() else -1
)


Device set to use cuda:0


In [15]:
raw_dataset = load_dataset("knkarthick/samsum")


In [ ]:
#prediction
from transformers import pipeline

sample = raw_dataset["test"][0]["dialogue"]
reference = raw_dataset["test"][0]["summary"]

print("Dialogue:\n", sample)
print("\nReference:\n", reference)
print("\nPrediction:\n", pipe(sample, max_length=128, num_beams=8)[0]["summary_text"])


Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
 Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference:
 Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Prediction:
 Amanda can't find Betty's number. Larry called Betty last time they were at the park together. Hannah wants Amanda to text Larry. Amanda will text Larry.


In [19]:
!zip -r pegasus_export.zip /kaggle/working/pegasus-samsum-model /kaggle/working/pegasus-samsum-tokenizer


  adding: kaggle/working/pegasus-samsum-model/ (stored 0%)
  adding: kaggle/working/pegasus-samsum-model/generation_config.json (deflated 40%)
  adding: kaggle/working/pegasus-samsum-model/config.json (deflated 61%)
  adding: kaggle/working/pegasus-samsum-model/model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 7%)
  adding: kaggle/working/pegasus-samsum-tokenizer/ (stored 0%)
  adding: kaggle/working/pegasus-samsum-tokenizer/tokenizer_config.json (deflated 94%)
  adding: kaggle/working/pegasus-samsum-tokenizer/special_tokens_map.json (deflated 82%)
  adding: kaggle/working/pegasus-samsum-tokenizer/spiece.model (deflated 50%)
  adding: kaggle/working/pegasus-samsum-tokenizer/tokenizer.json (deflated 78%)


In [20]:
# from IPython.display import FileLink

# FileLink('/kaggle/working/pegasus_export.zip')


/kaggle/working/pegasus_export.zip

In [22]:
# !ls /kaggle/working | grep pegasus_export


pegasus_export.zip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
#prediction
gen_kwargs={"length penalty":0.8, "num_beams":8, "max_length":128}
